In [2]:
!pip install pretrainedmodels

    100% |████████████████████████████████| 61kB 2.3MB/s ta 0:00:011
  Stored in directory: /tmp/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
Successfully built pretrainedmodels
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
from fastai.vision import *
from fastai.utils import mem
from fastai.callbacks import ReduceLROnPlateauCallback, SaveModelCallback
from sklearn.metrics import f1_score
from fastai.vision.learner import model_meta
import pretrainedmodels
print('Make sure cuda is installed:', torch.cuda.is_available())
print('Make sure cudnn is enabled:', torch.backends.cudnn.enabled)
mem.gpu_mem_get()

ModuleNotFoundError: No module named 'pretrainedmodels'

In [ ]:
#!kaggle competitions download -c iwildcam-2019-fgvc6

In [ ]:
train = pd.read_csv('../input/train.csv')
train = train[['file_name', 'category_id']]
train.head()

In [ ]:
test = pd.read_csv('../input/test.csv')
test = test[['file_name']]

In [ ]:
PATH = '../input/'

In [ ]:
datatest = ImageList.from_df(test, path=PATH, cols=0, folder='test_images')

In [ ]:
def get_data(bs, size):
    return (ImageList.from_df(train, path=PATH, cols=0, folder='train_images')
     .split_by_rand_pct(0.2, seed=47)
     .label_from_df(cols=1)
     .transform(get_transforms(xtra_tfms=[pad(mode='reflection')]), size=size)
     .add_test(datatest)
     .databunch(bs=bs)) 

In [ ]:
data = get_data(128, 32)
#stats = data.batch_stats()
data.normalize(imagenet_stats)

In [ ]:
data.show_batch()

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1., gamma=1.):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
        return F_loss.mean()

loss_func = FocalLoss(gamma=1.)

In [ ]:
def senet154(pretrained=False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.senet154(pretrained=pretrained)
    return model

In [ ]:
_se_resnet_meta = {'cut': -3, 'split': lambda m: (m[0][3], m[1]) }
model_meta[senet154] = _se_resnet_meta

In [ ]:
learn = create_cnn(data, senet154,  ps=0.5, wd=1e-1, loss_func=loss_func, metrics=[FBeta()], pretrained=True).to_fp16().mixup()

learn.lr_find()

learn.recorder.plot(suggestion=True)

In [ ]:
lr = 4.79E-02

In [ ]:
RLR = ReduceLROnPlateauCallback(learn, monitor='f_beta',patience = 2)
SAVEML = SaveModelCallback(learn, every='improvement', monitor='f_beta', name='best')

In [ ]:
#learn.fit_one_cycle(5, lr, callbacks = [RLR, SAVEML])

In [ ]:
learn.recorder.plot_losses() 

In [ ]:
learn.save('se-1')

In [ ]:
learn.load('best')

In [ ]:
learn.unfreeze()

In [ ]:
#learn.fit_one_cycle(5, slice(1e-5,1e-3), callbacks = [RLR, SAVEML])

In [ ]:
learn.load('best')

In [ ]:
learn.save('se-2')

In [ ]:
learn.recorder.plot_losses() 

# Validation

In [ ]:
pred, y = learn.get_preds()
#pred, y = learn.TTA()

In [ ]:
f1_score = f1_score(y, np.argmax(pred.numpy(), 1), average='macro')  
f1_score

# Inference
https://www.kaggle.com/iafoss/pretrained-resnet34-with-rgby-0-460-public-lb

In [ ]:
#learn = learn.to_fp32()

In [ ]:
#pred_t, _ = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
#import os
#test_ids = [os.path.basename(f)[:-4] for f in learn.data.test_ds.items]
#subm = pd.read_csv('sample_submission.csv')
#orig_ids = list(subm['Id'])

In [ ]:
#pred_t2 = np.argmax(pred_t.numpy(), 1)

In [ ]:
def create_submission(orig_ids, test_ids, preds):
    preds_dict = dict((k, v) for k, v in zip(test_ids, preds))
    pred_cor = [preds_dict[id] for id in orig_ids]
    df = pd.DataFrame({'id':orig_ids,'Predicted':pred_cor})
    df.to_csv(f'submission_{f1_score}.csv', header=True, index=False)
    return df

In [ ]:
#sub = create_submission(orig_ids, test_ids, pred_t2)

In [ ]:
#sub.tail()

In [ ]:
#! kaggle competitions submit -c iwildcam-2019-fgvc6 -f submission_0.5804866967400385.csv -m densenet